# MTURK API

### Intro
Access Amazon Mechanical Turk API using Boto Python Library. The process is two-folds:
 - test functions in sandbox mode
 - execute functions in your actual account
 
### Set Up
 - `pip install boto` in virtual environment

 - Create a file `mturk_key.py` that contains:
```
ACCESS_ID = 'YOUR_AWS_ACCESS_ID'
SECRET_KEY = 'YOUR_AWS_SECRET_KEY'
```

NB: **Do not push `mturk_key.py`** to public repo unless you plan on sharing your AWS information with others.


### More info 
 - [Read more on setting up AWS account](http://docs.aws.amazon.com/AWSMechTurk/latest/AWSMechanicalTurkGettingStartedGuide/SetUp.html)
 - [Create Worker Account in Sandbox Mode](https://workersandbox.mturk.com/mturk/welcome)
 - [Create Requester Account in Sandbox Mode](https://requestersandbox.mturk.com/manage)
 
### Gotchas
 - You cannot email a worker if you have ***not approved/rejected them*** in a previous HIT. This exists to control email SPAMs.
 - In sandbox mode, you must approve/reject worker before they receive your emails.



# Libraries

In [395]:
import boto
from boto.mturk.connection import MTurkConnection
from boto.mturk.price import Price

import mturk_key

# Activate Sandbox?

In [396]:
SANDBOX_IS_ACTIVE = True

# Credentials

In [397]:
ACCESS_ID = mturk_key.ACCESS_ID
SECRET_KEY = mturk_key.SECRET_KEY
HOST = 'mechanicalturk.sandbox.amazonaws.com' if SANDBOX_IS_ACTIVE else 'mechanicalturk.amazonaws.com'

# Connect account

In [398]:
mturk = MTurkConnection(aws_access_key_id=ACCESS_ID, aws_secret_access_key=SECRET_KEY, host=HOST)
print mturk.get_account_balance()

if SANDBOX_IS_ACTIVE:
    print '***************************'
    print 'SANDBOX testing is active'
    print '***************************'

[$10,000.00]
***************************
SANDBOX testing is active
***************************


# Show all HITs

In [399]:
all_hits = mturk.get_all_hits()
for i,h in enumerate(all_hits):
    print '{}: {}\t {}\t {}...'.format(i+1, h.CreationTime, h.HITId, h.Title[:50])

1: 2017-03-06T20:39:52Z	 3R16PJFTS3RRMPF63QOG9YDGY3YK4C	 sandy survey...


# Fetch assignments for specific HIT

In [400]:
hit = '3P0I4CQYVY7MLIJ5BOROBWZYF9FOWK'

if SANDBOX_IS_ACTIVE:
    hit = '3R16PJFTS3RRMPF63QOG9YDGY3YK4C'

cols = ['WorkerId', 'SubmitTime', 'AssignmentStatus']
cols_display = '\t\t'.join(cols)
print cols_display

total = 0
page = 1
enrolled_in_hit = []

while True:
    assignments = mturk.get_assignments(hit, page_number=page)
    if assignments == []: break
    for a in assignments:
        row = ''
        for c in cols:
            row += '{}\t\t'.format(getattr(a, c))
        print row
        enrolled_in_hit.append(a.WorkerId)
        total += 1
    page += 1


print
print '***************************************'
print 'Total workers enrolled in HIT: {}'.format(total)


WorkerId		SubmitTime		AssignmentStatus
AYMGZUNLQ3USS		2017-03-06T20:43:55Z		Approved		

***************************************
Total workers enrolled in HIT: 1


# Get workers of specific qualification

In [401]:
qual_name = 'vibration'

In [402]:
if SANDBOX_IS_ACTIVE:
    print '////////////////////////////////'
    print 'IGNORE BECAUSE SANDBOX IS ACTIVE'
    print '////////////////////////////////'
else:
    qual_id =  mturk.search_qualification_types(qual_name)[0].QualificationTypeId
    all_qual = mturk.get_qualifications_for_qualification_type(qualification_type_id=qual_id)
    workers_with_qual = [str(w.SubjectId) for w in all_qual]
    print 'No of workers with "{}" tag: {}'.format(qual_name, len(workers_with_qual))

////////////////////////////////
IGNORE BECAUSE SANDBOX IS ACTIVE
////////////////////////////////


# Get workers who have not enrolled

In [403]:
not_enrolled_in_hit = list(set(workers_with_qual) - set(enrolled_in_hit))
print 'No of workers NOT enrolled in HIT: {}'.format(len(not_enrolled_in_hit))

No of workers NOT enrolled in HIT: 45


# Prepare email to workers

In [404]:
def email_workers(all_ids, subject, message):
    for w in all_ids:
        mturk.notify_workers(w, subject, message)

In [411]:
subject = "More info about ongoing experiment"
message = """
Hi,

Some of you have reached out to me about enrolling in my experiment and I'd like to clarify a few details:

-- Be yourself.

-- Just do it.

-- Done!


Please reach out to me if you have any questions.

Thanks,
Researcher Name
"""

print '\t\t\t\t\t{0}'.format(subject)
print message

					More info about ongoing experiment

Hi,

Some of you have reached out to me about enrolling in my experiment and I'd like to clarify a few details:

-- Be yourself.

-- Just do it.

-- Done!


Please reach out to me if you have any questions.

Thanks,
Researcher Name



# Email workers

In [412]:
if SANDBOX_IS_ACTIVE:
    print '////////////////////////////////'
    print 'EMAIL SENT IN SANDBOX MODE'
    print '////////////////////////////////'
    sandbox_workers = enrolled_in_hit
    email_workers(sandbox_workers, subject, message)

else:
    response = raw_input("Are you sure you want to email them? Enter 'yes, go ahead and email': ")
    if response == 'yes, go ahead and email':
        email_workers(enrolled_in_hit, subject, message)
        print '****************************'
        print 'Email sent to all workers'
        print '****************************'
    else:
        print "Sorry, can't continue!"


////////////////////////////////
EMAIL SENT IN SANDBOX MODE
////////////////////////////////
